In [ ]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import easyocr
import sqlite3
import pandas as pd
from itertools import permutations
import re
from datetime import datetime

app = Flask(__name__)

# ====== المساعدة في تحويل الحاجات ======
def fix_year_smart(year):
    year = year.translate(str.maketrans("٠١٢٣٤٥٦٧٨٩", "0123456789"))
    for y in range(1990, 2026):
        if str(y) in set("".join(p) for p in permutations(year) if len(p) == 4):
            return str(y).translate(str.maketrans("0123456789", "٠١٢٣٤٥٦٧٨٩"))
    return year

def clean_part(part):
    return ''.join(c for c in part if not re.match(r"\s", c))[::-1] if re.search(r"\s", part) else part

def fix_arabic_date(text):
    if "_" not in text: return text
    day, month, year = map(clean_part, text.split("_"))
    return f"{day}_{month}_{fix_year_smart(year)}"

def normalize(text):
    return text.replace(" ", "")

def parse_arabic_date(arabic_date):
    arabic_numbers = str.maketrans('٠١٢٣٤٥٦٧٨٩', '0123456789')
    try:
        english_date = arabic_date.translate(arabic_numbers)
        return datetime.strptime(english_date, "%d_%m_%Y")
    except Exception:
        return None

# ====== البيانات المرجعية ======
reference_datasets = [
    ['٢٠٢١٥', '٥٢٠٢', 'سمر محمد ابراهيم حسن', '٠٢_١٢_٢٠١٧', '٢٠٧١٨٤', '٢٠٢٤', '١٢_١١_٢٠٢٧'],
    ['٦٨٨٢٦٩٥١', '٨٩٢٧١١٨', 'إبراهيم أحمد فؤاد عبد الكريم', '٠٢_١٢_٢٠٠٦', '٣١٤٥٩٢١', '٢٠٠٧', '٢٧_٠٤_٢٠٠١']
]

@app.route('/verify_id', methods=['POST'])
def verify_id():
    if 'image' not in request.files:
        return jsonify({"error": "No image part"}), 400

    file = request.files['image']
    npimg = np.frombuffer(file.read(), np.uint8)
    img = cv2.imdecode(npimg, cv2.IMREAD_COLOR)

    # Preprocess
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY_INV)

    # OCR
    reader = easyocr.Reader(['ar'])
    results = reader.readtext(thresh)

    # Filter
    texts = []
    for i, res in enumerate(results):
        text = normalize(res[1])
        if res[2] > 0.5:
            fixed = fix_arabic_date(text)
            texts.append(fixed)

    ocr_text = ' '.join(texts)

    # Verification
    verified = False
    expired = False
    message = "❌ ID غير موجود أو مزور ❌"

    for reference_data in reference_datasets:
        total_words = sum(len(ref.split()) for ref in reference_data)
        matched_words = sum(1 for ref in reference_data for word in ref.split() if word in ocr_text)

        match_ratio = matched_words / total_words

        if match_ratio >= 0.8:
            verified = True
            expiry_date_arabic = reference_data[-1]
            expiry_date = parse_arabic_date(expiry_date_arabic)
            today = datetime.today()

            if expiry_date and expiry_date < today:
                expired = True
                message = "⚠️ ID صحيح لكن منتهي الصلاحية - يحتاج تجديد 🔄"
            else:
                message = "✅ ID صالح وساري ✅"
            break

    return jsonify({
        "verified": verified,
        "expired": expired,
        "message": message
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.6:5000
Press CTRL+C to quit
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\LRR-\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
192.168.1.24 - - [14/Jun/2025 22:43:44] "POST /verify_id HTTP/1.1" 200 -
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
192.168.1.24 - - [14/Jun/2025 22:43:58] "POST /verify_id HTTP/1.1" 200 -
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
192.168.1.24 - - [14/Jun/2025 22:44:15] "POST /verify_id HTTP/1.1" 200 -
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster wi